## 7.1.1 函数式API 介绍

In [42]:
from tensorflow.keras import Input, layers

In [43]:
input_tensor = Input(shape=(32,))  #输入张量

In [44]:
dense = layers.Dense(32,activation='relu') #设定一层神经层

In [45]:
output_tensor = dense(input_tensor) 
#将input_tensor 放入神经层中进行运算 得到结果 output_tensor
#可以在一个张量上调用一个层,它会返回一个张量

In [48]:
from tensorflow.keras import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

seq_model = Sequential() #前面所学的 Sequential 模型
seq_model.add(layers.Dense(32, activation = 'relu',input_shape=(64,)))
seq_model.add(layers.Dense(32, activation = 'relu'))
seq_model.add(layers.Dense(10, activation = 'softmax'))


#以下 4 行为对应的函数式 API 实现
input_tensor = Input(shape=(64,)) 
x = layers.Dense(32, activation = 'relu')(input_tensor)
x = layers.Dense(32, activation = 'relu')(x)
output_tensor = layers.Dense(10, activation = 'softmax')(x)

model = Model(input_tensor,output_tensor) #Model 类将输入张量和输出张量转为模型

In [49]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_55 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_56 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_57 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy') #编译模型

In [52]:
import numpy as np
x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

model.fit(x_train, y_train, epochs=10,batch_size=128) #训练十轮

score = model.evaluate(x_train,y_train) #评估模型

Epoch 1/10
1000/1000 [==============================] - 0s 164us/sample - loss: 11.8162
Epoch 2/10
1000/1000 [==============================] - 0s 12us/sample - loss: 12.7320
Epoch 3/10
1000/1000 [==============================] - 0s 13us/sample - loss: 14.4509
Epoch 4/10
1000/1000 [==============================] - 0s 13us/sample - loss: 16.9897
Epoch 5/10
1000/1000 [==============================] - 0s 13us/sample - loss: 20.1216
Epoch 6/10
1000/1000 [==============================] - 0s 12us/sample - loss: 23.6277
Epoch 7/10
1000/1000 [==============================] - 0s 16us/sample - loss: 28.1449
Epoch 8/10
1000/1000 [==============================] - 0s 12us/sample - loss: 33.4308
Epoch 9/10
1000/1000 [==============================] - 0s 15us/sample - loss: 39.3006
Epoch 10/10
1000/1000 [==============================] - 0s 63us/sample - loss: 49.9331


## 7.1.2 多输入模型

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

In [4]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,),dtype='int32',name='text') # 文本输入是一个长度可变的整数序列。注意，你可以选择对输入进行命名

embedded_text = layers.Embedding(text_vocabulary_size,64)(text_input)  # 将输入嵌入长度为 64 的向量

encoded_text = layers.LSTM(32)(embedded_text) # 利用 LSTM 将向量编码为单个向量

question_input = Input(shape=(None,),
                      dtype='int32',
                      name='question')  # 对问题进行相同的处理（使用不同的层实例）

embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)

encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text,encoded_question],axis=-1) # 将编码后的问题和文本连接起来

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated) # 在上面添加一层 softmax 分类器

model = Model([text_input, question_input], answer) # 模型实例化时,指定两个输入和输出

model.compile('rmsprop','categorical_crossentropy',['acc'])

In [5]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [7]:
import numpy as np
import tensorflow
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples,max_length))  # 生成虚构的 numpy 数据
"""numpy.random.randint(low, high=None, size=None, dtype='l')
返回一个随机整型数，范围从低（包括）到高（不包括），即[low, high)。
如果没有写参数high的值，则返回[0,low)的值。"""
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = tensorflow.keras.utils.to_categorical(answers, answer_vocabulary_size) #回答是one-hot编码的，不是整数

model.fit([text, question], answers, epochs=10, batch_size=128) #使用输入组成的列表来拟合

model.fit({'text': text,'question': quesiton},answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 1s 1ms/sample - loss: 6.2146 - acc: 0.0040
Epoch 2/10
1000/1000 [==============================] - 1s 771us/sample - loss: 6.1950 - acc: 0.0300
Epoch 3/10
1000/1000 [==============================] - 1s 703us/sample - loss: 6.1190 - acc: 0.0070
Epoch 4/10
1000/1000 [==============================] - 1s 708us/sample - loss: 6.0377 - acc: 0.0090
Epoch 5/10
1000/1000 [==============================] - 1s 626us/sample - loss: 5.9685 - acc: 0.0090
Epoch 6/10
1000/1000 [==============================] - 1s 608us/sample - loss: 5.8951 - acc: 0.0100
Epoch 7/10
1000/1000 [==============================] - 1s 678us/sample - loss: 5.8097 - acc: 0.0110
Epoch 8/10
1000/1000 [==============================] - 1s 624us/sample - loss: 5.7478 - acc: 0.0110
Epoch 9/10
1000/1000 [==============================] - 1s 625us/sample - loss: 5.6454 - acc: 0.0250
Epoch 10/10
1000/1000 [==============================] - 1s 637us/sample - loss: 5.5762 - acc

NameError: name 'quesiton' is not defined